In [2]:
import pandas as pd
import mlflow
from Insurance.exception import InsuranceException
from mlflow.tracking import MlflowClient
from Insurance.utils import load_object

In [ ]:
class Experiments_evaluation:
    def __init__(self,experiment_name,run_name) :
        self.experiment_name=experiment_name
        self.run_name=run_name
        
        self.run_id=None
        self.model_name=model_name
    # Function to save run details to YAML file
    def save_run_details_to_yaml(self,run_id,model_name,model_uri,run_name):
        client = MlflowClient()
        run = client.get_run(run_id=run_id)
        
        report = {
            "Experiment": self.experiment_name,
            "Model_name":model_name,
            "run_name":run_name,
            "run_id": run.info.run_id,
            "experiment_id": run.info.experiment_id,
            "Parameters": run.data.params,
            "metrics": run.data.metrics,
            "Model_uri": model_uri
        }
            
        return report
            
            
    def get_best_model_run_id(self,experiment_name, metric_name):
        # Get the experiment ID
        experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
        
        # Retrieve runs and sort by the specified metric
        runs = mlflow.search_runs(experiment_ids=[experiment_id], filter_string='', order_by=[f"metrics.{metric_name} DESC"])
        
        if runs.empty:
            print("No runs found for the specified experiment and metric.")
            return None
        
        # Get the best run
        best_run = runs.iloc[0]
        best_run_id = best_run.run_id
        
        # Load the best model
      #  best_model = mlflow.sklearn.load_model(f"runs:/{best_run_id}/model")
        model_uri=(f"runs:/{best_run_id}/model")
        return model_uri,best_run_id

    def run_mlflow_experiment(self, R2_score, parameters, model_path):
        """
        Run an MLflow experiment, log metrics, parameters, and a model,
        and return the best model and its run ID based on the specified metric.
        
        Parameters:
        - experiment_name (str): Name of the MLflow experiment.
        - run_name (str): Name of the run.
        - R2_score (float): R2 score to be logged as a metric.
        - parameters (dict): Dictionary of parameters to be logged.
        - model: The machine learning model to be logged.
        
        Returns:
        - best_model (object): The best model from the experiment.
        - best_run_id (str): ID of the best run based on the specified metric.
        """
        # Create or get the experiment
        mlflow.set_experiment(self.experiment_name)
        
        # Start a run
        with mlflow.start_run(run_name=self.run_name):
            # Log metrics, params, and model
            mlflow.log_metric("R2_score", float(R2_score))
            mlflow.log_params(parameters)
             # Load your trained model
            model = load_object(model_path)
            mlflow.sklearn.log_model(model, "model")
        
        model_uri,best_run_id = self.get_best_model_run_id(metric_name='R2_score', experiment_name=self.experiment_name)
        
        print(f"Best model Run id: {best_run_id}")
        
        return   model_uri,best_run_id

In [ ]:
Experiments_evaluation(experiment_name=experiment_name,)